In [1]:
import sys
import os
import pandas as pd
import numpy as np
from Bio import SeqIO
import re

In [2]:
dataset = 'ZQ2022'

print('Loading raw data for', dataset, '...')
data = pd.read_csv('/Users/maryamkoddus/Documents/maryam-ko-QMUL-MSc-Project/01_input_data/raw_data/mc23462-sup-0002-supporting_tables.csv', header=2)
print('Raw data loaded.')
data

Loading raw data for ZQ2022 ...
Raw data loaded.


,Phosphopeptide,Sequence +/-7,Phosphosite,UniProt ID,Description,Gene Name,C1,C2,C3,N1,...,Unnamed: 246,Unnamed: 247,Unnamed: 248,Unnamed: 249,Unnamed: 250,Unnamed: 251,Unnamed: 252,Unnamed: 253,Unnamed: 254,Unnamed: 255
0,KLS(1)EADNRK,KEVTRKLpSEADNRKM,pS105,Q9UNL2,Translocon-associated protein subunit gamma,SSR3,0.914,1.113,1.038,1.031,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,RES(0.915)WS(0.082)S(0.003)APELPEELR,FQTERREpSWSSAPEL,pS598,Q96JH8,Ras-associating and dilute domain-containing p...,RADIL,0.682,1.017,0.986,1.305,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,IHSNSSSEEVSQELES(1)DDG,EVSQELEpSDDG,pS247,Q7Z2X4,"PTB-containing, cubilin and LRP1-interacting p...",PID1,0.728,0.907,0.908,1.080,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HS(0.004)T(0.022)PS(0.764)PT(0.209)R,EIKHSTPpSPTRYSLS,pS781,P49792,E3 SUMO-protein ligase RanBP2,RANBP2,1.270,1.145,1.610,0.726,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,GVHGS(1)FLAR,KGRGVHGpSFLARPSR,pS26,P29350,Tyrosine-protein phosphatase non-receptor type 6,PTPN6,1.393,1.417,1.520,0.541,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2491,LS(0.059)PS(0.937)PS(0.003)SR,ERLKLSPpSPSSRVTV,pS407,Q03252,Lamin-B2,LMNB2,0.890,1.027,0.968,1.061,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2492,T(0.008)AS(0.957)DFIT(0.035)K,KAALKTApSDFITKMD,pS363,P06396,Gelsolin,GSN,0.887,1.124,1.191,1.008,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2493,QPVELT(0.989)PT(0.011)DK,TRQPVELpTPTDKLFI,pT642,P05771,Protein kinase C beta type,PRKCB,1.000,1.048,1.075,1.062,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2494,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
print(data.columns.tolist())


['Phosphopeptide', 'Sequence +/-7', 'Phosphosite', 'UniProt ID', 'Description', 'Gene Name', 'C1', 'C2', 'C3', 'N1', 'N2', 'N3', 'C1/N1 Ratio', 'C2/N2 Ratio', 'C3/N3 Ratio', 'C/N Ratio', 'C/N P value', 'FDR', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38', 'Unnamed: 39', 'Unnamed: 40', 'Unnamed: 41', 'Unnamed: 42', 'Unnamed: 43', 'Unnamed: 44', 'Unnamed: 45', 'Unnamed: 46', 'Unnamed: 47', 'Unnamed: 48', 'Unnamed: 49', 'Unnamed: 50', 'Unnamed: 51', 'Unnamed: 52', 'Unnamed: 53', 'Unnamed: 54', 'Unnamed: 55', 'Unnamed: 56', 'Unnamed: 57', 'Unnamed: 58', 'Unnamed: 59', 'Unnamed: 60', 'Unnamed: 61', 'Unnamed: 62', 'Unnamed: 63', 'Unnamed: 64', 'Unnamed: 65', 'Unnamed: 66', 'Unnamed: 67', 'Unnamed: 68', 'Unnamed: 69', 'Unnamed: 7

In [4]:
# Filtering out semi-colons from 'Amino acid', 'Positions within proteins', and 'Gene names' columns
data = data[~data['Phosphosite'].str.contains(',', na=False)]
data = data[~data['Gene Name'].str.contains(',', na=False)]
data

,Phosphopeptide,Sequence +/-7,Phosphosite,UniProt ID,Description,Gene Name,C1,C2,C3,N1,...,Unnamed: 246,Unnamed: 247,Unnamed: 248,Unnamed: 249,Unnamed: 250,Unnamed: 251,Unnamed: 252,Unnamed: 253,Unnamed: 254,Unnamed: 255
0,KLS(1)EADNRK,KEVTRKLpSEADNRKM,pS105,Q9UNL2,Translocon-associated protein subunit gamma,SSR3,0.914,1.113,1.038,1.031,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,RES(0.915)WS(0.082)S(0.003)APELPEELR,FQTERREpSWSSAPEL,pS598,Q96JH8,Ras-associating and dilute domain-containing p...,RADIL,0.682,1.017,0.986,1.305,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,IHSNSSSEEVSQELES(1)DDG,EVSQELEpSDDG,pS247,Q7Z2X4,"PTB-containing, cubilin and LRP1-interacting p...",PID1,0.728,0.907,0.908,1.080,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HS(0.004)T(0.022)PS(0.764)PT(0.209)R,EIKHSTPpSPTRYSLS,pS781,P49792,E3 SUMO-protein ligase RanBP2,RANBP2,1.270,1.145,1.610,0.726,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,GVHGS(1)FLAR,KGRGVHGpSFLARPSR,pS26,P29350,Tyrosine-protein phosphatase non-receptor type 6,PTPN6,1.393,1.417,1.520,0.541,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2491,LS(0.059)PS(0.937)PS(0.003)SR,ERLKLSPpSPSSRVTV,pS407,Q03252,Lamin-B2,LMNB2,0.890,1.027,0.968,1.061,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2492,T(0.008)AS(0.957)DFIT(0.035)K,KAALKTApSDFITKMD,pS363,P06396,Gelsolin,GSN,0.887,1.124,1.191,1.008,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2493,QPVELT(0.989)PT(0.011)DK,TRQPVELpTPTDKLFI,pT642,P05771,Protein kinase C beta type,PRKCB,1.000,1.048,1.075,1.062,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2494,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
phospho_clean = data['Phosphosite'].dropna().astype(str)

# Extract amino acid (e.g., 'S' from 'pS270')
data['Amino Acid'] = phospho_clean.str[1]

# Extract position safely
data['Position'] = pd.to_numeric(phospho_clean.str[2:], errors='coerce')
print(data[['Phosphosite', 'Amino Acid', 'Position', 'Gene Name']].head(10))

   Phosphosite Amino Acid  Position Gene Name
0        pS105          S     105.0      SSR3
1        pS598          S     598.0     RADIL
2        pS247          S     247.0      PID1
3        pS781          S     781.0    RANBP2
4         pS26          S      26.0     PTPN6
5        pS280          S     280.0     MACF1
6        pT441          T     441.0     RIC8A
7        pT233          T     233.0     ASCC2
9        pS347          S     347.0    PCYT1A
10       pS767          S     767.0    PDGFRA


In [6]:
# filter data
data['Sequence +/-7'] = data['Sequence +/-7'].str.replace('_', '')
data

,Phosphopeptide,Sequence +/-7,Phosphosite,UniProt ID,Description,Gene Name,C1,C2,C3,N1,...,Unnamed: 248,Unnamed: 249,Unnamed: 250,Unnamed: 251,Unnamed: 252,Unnamed: 253,Unnamed: 254,Unnamed: 255,Amino Acid,Position
0,KLS(1)EADNRK,KEVTRKLpSEADNRKM,pS105,Q9UNL2,Translocon-associated protein subunit gamma,SSR3,0.914,1.113,1.038,1.031,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,105.0
1,RES(0.915)WS(0.082)S(0.003)APELPEELR,FQTERREpSWSSAPEL,pS598,Q96JH8,Ras-associating and dilute domain-containing p...,RADIL,0.682,1.017,0.986,1.305,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,598.0
2,IHSNSSSEEVSQELES(1)DDG,EVSQELEpSDDG,pS247,Q7Z2X4,"PTB-containing, cubilin and LRP1-interacting p...",PID1,0.728,0.907,0.908,1.080,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,247.0
3,HS(0.004)T(0.022)PS(0.764)PT(0.209)R,EIKHSTPpSPTRYSLS,pS781,P49792,E3 SUMO-protein ligase RanBP2,RANBP2,1.270,1.145,1.610,0.726,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,781.0
4,GVHGS(1)FLAR,KGRGVHGpSFLARPSR,pS26,P29350,Tyrosine-protein phosphatase non-receptor type 6,PTPN6,1.393,1.417,1.520,0.541,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,26.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2491,LS(0.059)PS(0.937)PS(0.003)SR,ERLKLSPpSPSSRVTV,pS407,Q03252,Lamin-B2,LMNB2,0.890,1.027,0.968,1.061,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,407.0
2492,T(0.008)AS(0.957)DFIT(0.035)K,KAALKTApSDFITKMD,pS363,P06396,Gelsolin,GSN,0.887,1.124,1.191,1.008,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,363.0
2493,QPVELT(0.989)PT(0.011)DK,TRQPVELpTPTDKLFI,pT642,P05771,Protein kinase C beta type,PRKCB,1.000,1.048,1.075,1.062,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,T,642.0
2494,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
def match_seq_to_genename(dataset, seq_column):
    '''
    Maps amino acid sequences to gene names using the loaded fasta file.
    
    args:
    =====
    dataset: <pd.Dataframe> with a column of amino acid sequences
    seq_column: <str> column name containing amino acid sequences
    
    out:
    ====
    dataset: <pd.Dataframe> with an additional column containing gene names
    '''    

    fasta_sequence = list(SeqIO.parse(open(f'/Users/maryamkoddus/Documents/maryam-ko-QMUL-MSc-Project/01_input_data/raw_data/UP000005640_9606.fasta'), "fasta"))
    
    
    gene_dict = {}
    
    # iterate over rows in seq_column
    for i in dataset[seq_column]:
        print(i)
        i_str = str(i)
        for seq_record in fasta_sequence:
            matches = re.findall(i_str, str(seq_record.seq))
            if matches:
                print(f"Match found for sequence: {seq_record}")
                gene_name_match = re.search(r"GN=(\w+)", seq_record.description)
                print('Gene name match:', gene_name_match)
                # gene_name_match = re.search("GN=(\w+)", seq_record.description)
                if gene_name_match:
                    gene_name = gene_name_match.group(1)
                    gene_dict[i] = gene_name
                    print(f"Match found: {i_str} -> {gene_name}")
                else: 
                    print(f"No gene name found in description for sequence: {i_str}")
    
    # map sequences to gene names           
    dataset['GeneName'] = dataset[seq_column].map(gene_dict) 
    print('Amino acid sequences matched to gene names.')
    return dataset 
    

In [8]:
data = match_seq_to_genename(data, 'Sequence +/-7')

KEVTRKLpSEADNRKM
FQTERREpSWSSAPEL
EVSQELEpSDDG
EIKHSTPpSPTRYSLS
KGRGVHGpSFLARPSR
AEDVDVPpSPDEKSVI
QYSEDEDpTDTDEYKE
LEERGRLpTPSDMPLL
GKTSPPCpSPANLSRH
SLYDRPApSYKKKSML
PHRYRPGpTVALREIR
AMKRALApSIDSKLNQ
KNRRDRNpSWGGFSEK
DRMQKEIpTALAPSTM
TLNLSQLpSFHRVDQK
QKPVVDHpSVRINSVG
AGVMITApSHNRKEDN
RSEHKVWpSPLVTEEG
HSGSGCRpSPSLGSDL
RLRLYHDpSTNIDKVN
SLGTGTApSPRTSLEV
RVLQQQGpSQERSPGL
SQRDLTHpSDSESSLH
DASPEAGpSGGGGVAL
AHSDKPGpSTSTASFR
DFCDREQpSLKETLVE
ANNQKVYpTPHDGKLV
RYKSYSPpYDMLESIR
RALVHQLpSNESRLSI
QKEYLSQpSCPSVSNV
MPYpSVGFREAD
SNMKRAApSLNYLNQP
KAVSYHApSGHSVAYK
GSDRLHMpSLQQGKSS
LIRNKFGpSADNIAHL
EKAEAPSpSPDVAPAG
AANYSSTpSTRREHVK
RLLNLQDpSDSEECTS
GPGEGVLpTLRAKPPP
PRNKRSRpSPLELEPE
PSSLRRVpTIASLPRN
VLAARSDpSWSLANLS
AKSSSHLpSPSEEVRC
LLIVQDApSERAALIP
HLSDNDApSDVE
GCSEQVLpTVLKTVSL
RRFHVQLpSNPKFGAH
EAALPPVpSPLKAALS
LNDRFANpYIDKVRFL
DRNVIPDpTPPSTPLV
KRHCRSLpSVPVDLSR
GVLSRTSpSIQPSFRH
QERLLGGpTRTPINDA
SKEVLMEpSPPDYSAA
PPGGTHGpSLKTSAMR
QKYWGPApSPTHKLPP
SRRSLTNpSHLEKKKC
VNSHLPGpSPKQNRST
APVTPPPpSVGGSRQR
QPGLRQPpS

In [9]:
data = data.dropna(subset=['Sequence +/-7'])


In [10]:
print(data[['Amino Acid', 'Position', 'Phosphosite']].head())


  Amino Acid  Position Phosphosite
0          S     105.0       pS105
1          S     598.0       pS598
2          S     247.0       pS247
3          S     781.0       pS781
4          S      26.0        pS26


In [11]:
data['Phosphosite'] = data['Amino Acid'].astype(str) + '(' + data['Position'].astype(int).astype(str) + ')'
data

/var/folders/1v/y_z33m9d25d2tszlfrvnp2300000gn/T/ipykernel_12320/601792056.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Phosphosite'] = data['Amino Acid'].astype(str) + '(' + data['Position'].astype(int).astype(str) + ')'


,Phosphopeptide,Sequence +/-7,Phosphosite,UniProt ID,Description,Gene Name,C1,C2,C3,N1,...,Unnamed: 249,Unnamed: 250,Unnamed: 251,Unnamed: 252,Unnamed: 253,Unnamed: 254,Unnamed: 255,Amino Acid,Position,GeneName
0,KLS(1)EADNRK,KEVTRKLpSEADNRKM,S(105),Q9UNL2,Translocon-associated protein subunit gamma,SSR3,0.914,1.113,1.038,1.031,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,105.0,NaN
1,RES(0.915)WS(0.082)S(0.003)APELPEELR,FQTERREpSWSSAPEL,S(598),Q96JH8,Ras-associating and dilute domain-containing p...,RADIL,0.682,1.017,0.986,1.305,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,598.0,NaN
2,IHSNSSSEEVSQELES(1)DDG,EVSQELEpSDDG,S(247),Q7Z2X4,"PTB-containing, cubilin and LRP1-interacting p...",PID1,0.728,0.907,0.908,1.080,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,247.0,NaN
3,HS(0.004)T(0.022)PS(0.764)PT(0.209)R,EIKHSTPpSPTRYSLS,S(781),P49792,E3 SUMO-protein ligase RanBP2,RANBP2,1.270,1.145,1.610,0.726,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,781.0,NaN
4,GVHGS(1)FLAR,KGRGVHGpSFLARPSR,S(26),P29350,Tyrosine-protein phosphatase non-receptor type 6,PTPN6,1.393,1.417,1.520,0.541,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,26.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2489,MNS(0.999)LT(0.001)FK,KVFERMNpSLTFKKSK,S(1383),Q96N67,Dedicator of cytokinesis protein 7,DOCK7,1.319,1.498,1.455,0.636,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,1383.0,NaN
2490,DTTIINHS(1)R,DTTIINHpSRNGSYGE,S(720),Q8IY63,Angiomotin-like protein 1,AMOTL1,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,720.0,NaN
2491,LS(0.059)PS(0.937)PS(0.003)SR,ERLKLSPpSPSSRVTV,S(407),Q03252,Lamin-B2,LMNB2,0.890,1.027,0.968,1.061,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,407.0,NaN
2492,T(0.008)AS(0.957)DFIT(0.035)K,KAALKTApSDFITKMD,S(363),P06396,Gelsolin,GSN,0.887,1.124,1.191,1.008,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,363.0,NaN


In [12]:
keepcols = ['Phosphosite', 'Gene Name'] + [col for col in data.columns if col in ['C1', 'C2', 'C3', 'N1', 'N2', 'N3']]
data = data[keepcols]
data

,Phosphosite,Gene Name,C1,C2,C3,N1,N2,N3
0,S(105),SSR3,0.914,1.113,1.038,1.031,0.954,0.951
1,S(598),RADIL,0.682,1.017,0.986,1.305,0.979,1.030
2,S(247),PID1,0.728,0.907,0.908,1.080,1.132,1.245
3,S(781),RANBP2,1.270,1.145,1.610,0.726,0.615,0.634
4,S(26),PTPN6,1.393,1.417,1.520,0.541,0.588,0.540
...,...,...,...,...,...,...,...,...
2489,S(1383),DOCK7,1.319,1.498,1.455,0.636,0.481,0.611
2490,S(720),AMOTL1,NaN,NaN,NaN,NaN,NaN,NaN
2491,S(407),LMNB2,0.890,1.027,0.968,1.061,1.134,0.920
2492,S(363),GSN,0.887,1.124,1.191,1.008,0.932,0.859


In [13]:
ratio_columns = [col for col in data.columns if col in ['C1', 'C2', 'C3', 'N1', 'N2', 'N3']]
data[ratio_columns] = data[ratio_columns].apply(pd.to_numeric, errors='coerce')

/var/folders/1v/y_z33m9d25d2tszlfrvnp2300000gn/T/ipykernel_12320/619714419.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[ratio_columns] = data[ratio_columns].apply(pd.to_numeric, errors='coerce')


In [14]:
def log2_transform(dataset):
    '''
    Log2 transform a dataset.
    
    args:
    =====
    dataset: <pd.DataFrame>
    
    out:
    ====
    dataset: <pd.DataFrame> with log2 transformed values
    '''
    cols_to_transform = [col for col in data.columns if col in ['C1', 'C2', 'C3', 'N1', 'N2', 'N3']]
    dataset[cols_to_transform] = np.log2(dataset[cols_to_transform])
    print('Data has been log2 transformed.')
    return dataset

In [15]:
# Apply the log2 transformation
data = log2_transform(data)
print(f"DataFrame after log2 transformation:\n{data}")
data

Data has been log2 transformed.
DataFrame after log2 transformation:
     Phosphosite Gene Name        C1        C2        C3        N1        N2  \
0         S(105)      SSR3 -0.129734  0.154454  0.053806  0.044044 -0.067939   
1         S(598)     RADIL -0.552156  0.024320 -0.020340  0.384050 -0.030619   
2         S(247)      PID1 -0.457990 -0.140826 -0.139236  0.111031  0.178874   
3         S(781)    RANBP2  0.344828  0.195348  0.687061 -0.461959 -0.701342   
4          S(26)     PTPN6  0.478195  0.502840  0.604071 -0.886300 -0.766112   
...          ...       ...       ...       ...       ...       ...       ...   
2489     S(1383)     DOCK7  0.399445  0.583038  0.541019 -0.652901 -1.055891   
2490      S(720)    AMOTL1       NaN       NaN       NaN       NaN       NaN   
2491      S(407)     LMNB2 -0.168123  0.038436 -0.046921  0.085425  0.181421   
2492      S(363)       GSN -0.172994  0.168642  0.252173  0.011496 -0.101598   
2493      T(642)     PRKCB  0.000000  0.067639  0.1

/var/folders/1v/y_z33m9d25d2tszlfrvnp2300000gn/T/ipykernel_12320/1173117191.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[cols_to_transform] = np.log2(dataset[cols_to_transform])


,Phosphosite,Gene Name,C1,C2,C3,N1,N2,N3
0,S(105),SSR3,-0.129734,0.154454,0.053806,0.044044,-0.067939,-0.072483
1,S(598),RADIL,-0.552156,0.024320,-0.020340,0.384050,-0.030619,0.042644
2,S(247),PID1,-0.457990,-0.140826,-0.139236,0.111031,0.178874,0.316146
3,S(781),RANBP2,0.344828,0.195348,0.687061,-0.461959,-0.701342,-0.657445
4,S(26),PTPN6,0.478195,0.502840,0.604071,-0.886300,-0.766112,-0.888969
...,...,...,...,...,...,...,...,...
2489,S(1383),DOCK7,0.399445,0.583038,0.541019,-0.652901,-1.055891,-0.710756
2490,S(720),AMOTL1,NaN,NaN,NaN,NaN,NaN,NaN
2491,S(407),LMNB2,-0.168123,0.038436,-0.046921,0.085425,0.181421,-0.120294
2492,S(363),GSN,-0.172994,0.168642,0.252173,0.011496,-0.101598,-0.219270


In [16]:
def create_phos_ID(dataset):
    '''
    Concatenates GeneName and Phosphosite columns.
    
    args:
    =====
    dataset: <pd.Dataframe> with columns 'GeneName' and 'Phosphosite'
    
    out:
    ====
    dataset: <pd.Dataframe> with 'phosphosite_ID' column and 'GeneName' + 'Phosphosite' columns dropped
    '''
    dataset.loc[:, 'phosphosite_ID'] = dataset['Gene Name'].astype(str) + '_' + dataset['Phosphosite'].astype(str)
    dataset = dataset.drop(columns=['Phosphosite', 'Gene Name'])
    print('Phosphosite IDs created.')
    return dataset

data = create_phos_ID(data) # call function to create phosphosite_ID column

print('Phosphosite IDs created.')

/var/folders/1v/y_z33m9d25d2tszlfrvnp2300000gn/T/ipykernel_12320/19416799.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.loc[:, 'phosphosite_ID'] = dataset['Gene Name'].astype(str) + '_' + dataset['Phosphosite'].astype(str)


Phosphosite IDs created.
Phosphosite IDs created.


In [17]:
def clean_phosID_col(data):
    data = data[~data.phosphosite_ID.str.contains('nan', case=False, na=False)]
    data = data[~data.phosphosite_ID.str.contains(';', case=False, na=False)]
    data = data[~data.phosphosite_ID.str.contains('-', case=False, na=False)]

    # Add this line to remove decimals from phosphosite_ID (e.g., S123.0 -> S123)
    data['phosphosite_ID'] = data['phosphosite_ID'].apply(lambda x: re.sub(r'\((\d+)\.0+\)', r'(\1)', x))
    
    data_grouped = data.groupby(by='phosphosite_ID')
    
    if len(data) != len(data_grouped):
        numeric_cols = data.select_dtypes(include=[np.number]).columns.tolist()
        non_numeric_cols = data.columns.difference(numeric_cols + ['phosphosite_ID']).tolist()
        data_numeric = data_grouped[numeric_cols].mean()
        data_categorical = data_grouped[non_numeric_cols].first().reset_index()
        
        # Merge numeric and non-numeric parts
        data = pd.merge(data_categorical, data_numeric, on='phosphosite_ID')
        print('Phosphosites with multiple measurements have been averaged')
    else:
        print('There are no phosphosites with multiple measurements')

    # Replace inf values with NaNs
    data = data.replace([np.inf, -np.inf], np.nan)
    
    # Ensure phosphosite_ID is first column
    if data.columns[0] != 'phosphosite_ID':
        phosphosite_ID = data.pop('phosphosite_ID')
        data.insert(0, 'phosphosite_ID', phosphosite_ID)

    return data

In [18]:
data = clean_phosID_col(data)
print("After cleaning phosphosite_ID column:")
data

There are no phosphosites with multiple measurements
After cleaning phosphosite_ID column:


,phosphosite_ID,C1,C2,C3,N1,N2,N3
0,SSR3_S(105),-0.129734,0.154454,0.053806,0.044044,-0.067939,-0.072483
1,RADIL_S(598),-0.552156,0.024320,-0.020340,0.384050,-0.030619,0.042644
2,PID1_S(247),-0.457990,-0.140826,-0.139236,0.111031,0.178874,0.316146
3,RANBP2_S(781),0.344828,0.195348,0.687061,-0.461959,-0.701342,-0.657445
4,PTPN6_S(26),0.478195,0.502840,0.604071,-0.886300,-0.766112,-0.888969
...,...,...,...,...,...,...,...
2489,DOCK7_S(1383),0.399445,0.583038,0.541019,-0.652901,-1.055891,-0.710756
2490,AMOTL1_S(720),NaN,NaN,NaN,NaN,NaN,NaN
2491,LMNB2_S(407),-0.168123,0.038436,-0.046921,0.085425,0.181421,-0.120294
2492,GSN_S(363),-0.172994,0.168642,0.252173,0.011496,-0.101598,-0.219270


In [19]:
data.to_csv(f'/Users/maryamkoddus/Documents/maryam-ko-QMUL-MSc-Project/01_input_data/PreprocessedDatasets/ZQ2022.csv', index=False)


print(dataset, 'has been saved to CSV successfully!', data)     

ZQ2022 has been saved to CSV successfully!      phosphosite_ID        C1        C2        C3        N1        N2  \
0       SSR3_S(105) -0.129734  0.154454  0.053806  0.044044 -0.067939   
1      RADIL_S(598) -0.552156  0.024320 -0.020340  0.384050 -0.030619   
2       PID1_S(247) -0.457990 -0.140826 -0.139236  0.111031  0.178874   
3     RANBP2_S(781)  0.344828  0.195348  0.687061 -0.461959 -0.701342   
4       PTPN6_S(26)  0.478195  0.502840  0.604071 -0.886300 -0.766112   
...             ...       ...       ...       ...       ...       ...   
2489  DOCK7_S(1383)  0.399445  0.583038  0.541019 -0.652901 -1.055891   
2490  AMOTL1_S(720)       NaN       NaN       NaN       NaN       NaN   
2491   LMNB2_S(407) -0.168123  0.038436 -0.046921  0.085425  0.181421   
2492     GSN_S(363) -0.172994  0.168642  0.252173  0.011496 -0.101598   
2493   PRKCB_T(642)  0.000000  0.067639  0.104337  0.086784 -0.256700   

            N3  
0    -0.072483  
1     0.042644  
2     0.316146  
3    -0.6574